# Disordered RP generator

Script to generate (disordered) Ruddlesden Popper structure with any n-value. 
Multiple n-values will create a disordered Ruddlesden Popper structure.

For example:
- `rp_generator -n 5 6` will create 5 perovskite layers + rocksalt layer + 6 perovskite layers + rocksalt layer.
- `rp_generator -n 3` will create the standard RP material A$_4$B$_3$X$_{10}$.

Creates phase in a high-symmetry tetragonal structure.
After creating this structure you will need to relax to find the equilibrium structure. 
Importantly, you must allow the atomic positions **and** cell volume to relax.
You most likely also want to use the `rattle` option to break symmetry and allow for distortions to lower symmetry.

Use with caution as this hasn't been thoroughly tested.

In [32]:
import ase
from ase import Atoms
from ase import io
import numpy as np

In [26]:
def perovskite_atoms(A,B,X,cell_length):
    """Returns single perovskite layer"""
    
    return Atoms([A,B,X,X,X], 
           scaled_positions=[(0.5,0.5,0.5),(0,0,0),(0.5,0,0),(0,0,0.5),(0,0.5,0)],
           cell=[cell_length,cell_length,cell_length,90,90,90])

def rocksalt_atoms(A,X,cell_length):
    """Returns rocksalt layer"""
    
    return Atoms([A,X], 
           scaled_positions=[(0,0,0),(0.5,0.5,0)],
           cell=[cell_length,cell_length,1,90,90,90])

def save_structure(atoms_object,n_array,A,B,X):
    """Save structure as cif"""

    ase.io.write('RP_{}_{}.cif'.format(''.join(map(str,n_array)),A+B+X), atoms_object)

def create_mixed_rp(n_array,A='Ba',B='Zr',X='S',cell_length=5,rattle=False,save=True):
    """Returns (disordered) RP phase as an ASE atoms object.
    """

    # if the number of n-values is odd then we must double the array to make even.
    # This is to ensure that when pbc is applied
    # the neighbouring perovskite slabs are correctly offset from one-another
    if len(n_array) % 2 == 1:
        n_array = np.concatenate((n_array,n_array))

    # we need to create structure and apply pbc before adding in the perovskite and rocksalt layers
    total_length = np.sum(n_array)*1*cell_length + len(n_array)*0.5*cell_length 
    rp_structure = Atoms(cell=[cell_length,cell_length,total_length,90,90,90],pbc=[True,True,True])

    # create perovskite and rocksalt layer
    perovskite_layer = perovskite_atoms(A,B,X,cell_length)
    rocksalt_layer = rocksalt_atoms(A,X,cell_length)

    # for each slab of perovskite
    for number_of_layers in n_array:

        # for each perovskite layer within the slab
        for _ in range(number_of_layers):

            # add a perovskite layer to the rp_structure
            rp_structure += perovskite_layer
            
            # translate the perovskite and rocksalt layers along c-axis ready for next insertion
            perovskite_layer.translate(np.dot([0,0,1],perovskite_layer.cell))
            rocksalt_layer.translate(np.dot([0,0,1],perovskite_layer.cell))

        # after creating perovksite slab, add a rocksalt layer
        rp_structure += rocksalt_layer
        
        # translate the perovskite and rocksalt layers along c-axis ready for next insertion
        # also translate the rocksalt and perovskite layers in the ab-plane
        rocksalt_layer.translate(np.dot([0.5,0.5,0.5],perovskite_layer.cell))
        perovskite_layer.translate(np.dot([0.5,0.5,0.5],perovskite_layer.cell))

    if rattle:
        rp_structure.rattle(stdev=0.02, seed=1)

    if save:
        save_structure(rp_structure,n_array,A,B,X)

    return rp_structure

In [31]:
atoms = create_mixed_rp(np.array([1]),rattle=False)